In [1]:
# 模型训练-验证

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

In [3]:
train_path = r'E:\DataSet\Tianchi\zhengqi\zhengqi_train.txt'
test_path = r'E:\DataSet\Tianchi\zhengqi\zhengqi_test.txt'

train_data = pd.read_csv(train_path, delimiter="\t", encoding='utf-8')
test_data = pd.read_csv(test_path, delimiter="\t", encoding='utf-8')

In [ ]:
# 数据归一化处理